In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
from my_utils.preprocess import DataPreprocess
from my_utils.nfold_test import NFoldModel
from my_utils.dataloader import Workout_dataset
from my_utils.model import CNN_RNN_Resnet

In [2]:
# seed 고정
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(seed)
np.random.seed(seed)
rn.seed(seed)

In [3]:
features_path = './data/train_features.csv'
labels_path = './data/train_labels.csv'

preprocess = DataPreprocess(
    folds=10,seed=seed,acc_norm=10,gy_norm=2000,
    features_path=features_path,labels_path=labels_path)

In [4]:
preprocess.labels_to_csv()
preprocess.features_to_npy()

In [5]:
nfold_model = NFoldModel(batch_size=64, valid_ratio=4, lr=0.001, early_stop_patience=30, seed=42)
nfold_model.Model_class = CNN_RNN_Resnet
nfold_model.Workout_dataset = Workout_dataset
nfold_model.model_parameter = dict(leakyrelu_alpha = 0.1, 
    input_kernels = 20, input_kernel_width = 3, input_regularize_coeff=0.001,
    res_kernels = 60, res_kernel_width = 3, res_regularize_coeff=0.1, res_num = 5
    )
nfold_model.file_base_name = 'CNN_RNN_Resnet'

In [6]:
fold_list = [1,3,5,7,9]

In [7]:
nfold_model.nfold_train(fold_list)

009: val_loss improved from 7.10460 to 6.53819, saving model to ./save/CNN_RNN_Resnet_fold_7.hdf5
Epoch 10/2000
35/35 [==============================] - 16s 446ms/step - loss: 4.4313 - accuracy: 0.6531 - val_loss: 5.9267 - val_accuracy: 0.4911

Epoch 00010: val_loss improved from 6.53819 to 5.92669, saving model to ./save/CNN_RNN_Resnet_fold_7.hdf5
Epoch 11/2000
35/35 [==============================] - 16s 443ms/step - loss: 4.0132 - accuracy: 0.6625 - val_loss: 5.2243 - val_accuracy: 0.5089

Epoch 00011: val_loss improved from 5.92669 to 5.22431, saving model to ./save/CNN_RNN_Resnet_fold_7.hdf5
Epoch 12/2000
35/35 [==============================] - 16s 444ms/step - loss: 3.7379 - accuracy: 0.6710 - val_loss: 5.0525 - val_accuracy: 0.4821

Epoch 00012: val_loss improved from 5.22431 to 5.05250, saving model to ./save/CNN_RNN_Resnet_fold_7.hdf5
Epoch 13/2000
35/35 [==============================] - 16s 452ms/step - loss: 3.4233 - accuracy: 0.6790 - val_loss: 6.1183 - val_accuracy: 0.19

In [10]:
test_batch_size = 625
nfold_model.evaluate_test(test_batch_size,1)
nfold_model.evaluate_test(test_batch_size,3)
nfold_model.evaluate_test(test_batch_size,5)
nfold_model.evaluate_test(test_batch_size,7)
nfold_model.evaluate_test(test_batch_size,9)

1/1 [==============================] - 19s 19s/step - loss: 0.9968 - accuracy: 0.8736


In [12]:
import time
pred1 = nfold_model.predict_test(test_batch_size,1)
time.sleep(2)
pred3 = nfold_model.predict_test(test_batch_size,3)
time.sleep(2)
pred5 = nfold_model.predict_test(test_batch_size,5)
time.sleep(2)
pred7 = nfold_model.predict_test(test_batch_size,7)
time.sleep(2)
pred9 = nfold_model.predict_test(test_batch_size,9)
time.sleep(2)
predict = (pred1+pred3+pred5+pred7+pred9)/5

In [13]:
score_dict = nfold_model.nfold_score(preprocess.y_test,predict,average='macro')
print(score_dict)

{'accuracy': 0.88, 'recall': 0.7600440650807346, 'precision': 0.7986742577189156, 'f1': 0.7660757544773337}
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
score_dict = nfold_model.nfold_score(preprocess.y_test,predict,average='weighted')
print(score_dict)

{'accuracy': 0.88, 'recall': 0.88, 'precision': 0.8749504105090311, 'f1': 0.8714353725306339}
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
score_dict = nfold_model.nfold_score(preprocess.y_test,predict,average='micro')
print(score_dict)

{'accuracy': 0.88, 'recall': 0.88, 'precision': 0.88, 'f1': 0.88}
